# 一、课堂代码复刻

In [108]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [109]:
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col = 0)

In [110]:
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


### 语料预处理 

#### 剔除符号与数字

In [111]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)[:100]

'国债地产行业重磅利好提振风险偏好期债低开低走国债期货全线收跌年期主力合约跌年期主力合约跌年期主力合约跌三大主力合约均创逾一年收盘新低行情解读公开市场方面央行开展了亿元天期逆回购操作中标利率亿元逆回购到'

#### 分词

In [112]:
# 加载中文停用词词典，可个性化设置
# stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题/微博内容'][0]
clean_text(test)

'    国债 ： 地产 行业 重磅 利好 提振 风险 偏好 ， 期债 低开 低 走           国债 期货 全线 收跌 ， 10 年期 主力 合约 跌 0.37% ， 5 年期 主力 合约 跌 0.24% ， 2 年期 主力 合约 跌 0.11% ， 三大 主力 合约 均 创 逾 一年 收盘 新低 。         行情 解读 ：         1 ， 公开市场 方面 ， 央行 开展 了 800 亿元 7 天期 逆 回购 操作 ， 中标 利率 2.0% ， 20 亿元 逆 回购 到期 ， 因此 当日 净 投放 780 亿元 。         2 ， 资金面 方面 ， 央行 公开市场 逆 回购 继续 加力 ， 不过 银行 间 市场 周二 资金 供给 趋于 收敛 ， 七天 回购 加权 利率 进一步 上行 。 长期 资金 方面 ， 全国 和 主要 股份制 银行 一年期 同业 存单 报价 仍 维持 在 2.45% 水平 ， 不过 目前 尚无 成交量 配合 。         3 ， 房地产 行业 再度 迎来 重磅 利好 。 证监会 宣布 在 涉房 企业 股权 融资 方面 调整 优化 五项 措施 ， 并 自即日起 施行 。 随着 证监会 支持 房企 股权 融资 这 “ 第三支 箭 ” 正式 发射 ， 信贷 、 债券 、 股权 三个 融资 渠道 “ 三箭 齐发 ” ， 合力 推动 房地产 融资 。 — — “ 第三支 箭 ” 快速 落地 ， 或 将 为 房地产 行业 注入 较 大规模 的 增量 资金 和 资产 ， 促进 行业 平稳 健康 发展 。 在 短短 20 天 的 时间 内 政策 端 “ 三箭 齐发 ” 强势 托底 ， 提振 各方 的 信心 。 看好 全 行业 的 信用 复苏 以及 在 因城 施策 空间 逐步 打开 下 的 销售 回暖 。         4 ， 国务院 联防 联控 机制 11 月 29 日 15 : 00 召开 新闻 发布会 。 针对 “ 考虑 到 最近 个别 地方 的 不满情绪 ， 政府 是否 会 重新考虑 疫情 应对 政策 ” 的 问题 ， 国家 卫生 健康 委 新闻 发言人 、 宣传司 副司长 米锋 称 ， 疫情 发生 以来 ， 中国政府 一直 根据 病毒 变异 的 特点 和 临床 治疗 的 实践 认识 ， 边 防控 、 边 研究 

In [113]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他,"[国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 低, 走,..."
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西,"[喜迎, 二十大, 忠诚, 保平安]"
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建,"[注意, 事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告,..."
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京,"[近日, 千年, 大计, 雄安, 新区, 迎来, 五周岁, 生日, 从, 一张白纸, 到, ..."
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东,"[樊振东, 牛, 逼]"
...,...,...,...,...,...,...,...,...,...,...
95,14034,老公好漂亮[舔屏],0,0,0,aa7af97ae98cbe8983e559b948ecfabf,825,191,北京,"[老公, 好, 漂亮, 舔, 屏]"
96,35249,恭喜@张小娜呀娜 1名用户获得【小飞T】。C官方唯一抽奖工具@C抽奖平台 对本次抽奖进行监督...,0,0,2,f538513e5801c275cbcf285517a8ee62,51709,20,北京,"[恭喜, 张小娜, 呀, 娜, 名, 用户, 获得, 小飞, 官方, 唯一, 抽奖, 工具,..."
97,39598,接不动了，居民部门负债率提升空间太有限了。现在动辄几万一平米，总价高，月供高，预期转差，还怎...,0,0,0,2e76c39b6665a0b6f5bdc4c35f36f573,0,59,四川,"[接不动, 了, 居民, 部门, 负债率, 提升, 空间, 太, 有限, 了, 现在, 动辄..."
98,48977,目前电子烟政策已从制定阶段逐渐走向实施阶段，而主要的电子烟公司股价下跌幅度高达70%-...,0,0,0,NaN,0,0,北京,"[目前, 电子, 烟, 政策, 已, 从, 制定, 阶段, 逐渐, 走向, 实施, 阶段, ..."


### LDA

In [114]:
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

In [115]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [116]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.034*"的" + 0.013*"电子" + 0.012*"烟" + 0.010*"年" + 0.009*"在"')
(1, '0.037*"的" + 0.007*"在" + 0.005*"了" + 0.005*"和" + 0.005*"期货"')
(2, '0.018*"的" + 0.016*"月" + 0.013*"万吨" + 0.009*"市场" + 0.008*"日"')
(3, '0.027*"的" + 0.013*"了" + 0.007*"在" + 0.005*"也" + 0.004*"有"')
(4, '0.018*"考生" + 0.011*"的" + 0.011*"考试" + 0.010*"考场" + 0.007*"考点"')


In [117]:
df['微博内容分词'].iloc[0]

['国债',
 '地产',
 '行业',
 '重磅',
 '利好',
 '提振',
 '风险',
 '偏好',
 '期债',
 '低开',
 '低',
 '走',
 '国债',
 '期货',
 '全线',
 '收跌',
 '年期',
 '主力',
 '合约',
 '跌',
 '年期',
 '主力',
 '合约',
 '跌',
 '年期',
 '主力',
 '合约',
 '跌三大',
 '主力',
 '合约',
 '均',
 '创',
 '逾',
 '一年',
 '收盘',
 '新低',
 '行情',
 '解读',
 '公开市场',
 '方面',
 '央行',
 '开展',
 '了',
 '亿元',
 '天期',
 '逆',
 '回购',
 '操作',
 '中标',
 '利率',
 '亿元',
 '逆',
 '回购',
 '到期',
 '因此',
 '当日',
 '净',
 '投放',
 '亿元',
 '资金面',
 '方面',
 '央行',
 '公开市场',
 '逆',
 '回购',
 '继续',
 '加力',
 '不过',
 '银行',
 '间',
 '市场',
 '周二',
 '资金',
 '供给',
 '趋于',
 '收敛',
 '七天',
 '回购',
 '加权',
 '利率',
 '进一步',
 '上行',
 '长期',
 '资金',
 '方面',
 '全国',
 '和',
 '主要',
 '股份制',
 '银行',
 '一年期',
 '同业',
 '存单',
 '报价',
 '仍',
 '维持',
 '在',
 '水平',
 '不过',
 '目前',
 '尚无',
 '成交量',
 '配合',
 '房地产',
 '行业',
 '再度',
 '迎来',
 '重磅',
 '利好',
 '证监会',
 '宣布',
 '在',
 '涉房',
 '企业',
 '股权',
 '融资',
 '方面',
 '调整',
 '优化',
 '五项',
 '措施',
 '并',
 '自即日起',
 '施行',
 '随着',
 '证监会',
 '支持',
 '房企',
 '股权',
 '融资',
 '这',
 '第三支',
 '箭',
 '正式',
 '发射',
 '信贷',
 '债券',
 '股权',
 '三个',
 '融资',
 '渠道',
 '三箭',
 '齐发',
 '合

In [118]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9750389456748962	 
Topic: 0.037*"的" + 0.007*"在" + 0.005*"了" + 0.005*"和" + 0.005*"期货" + 0.005*"月" + 0.004*"为" + 0.004*"新" + 0.004*"公司" + 0.004*"等"

Score: 0.02453048713505268	 
Topic: 0.018*"的" + 0.016*"月" + 0.013*"万吨" + 0.009*"市场" + 0.008*"日" + 0.008*"库存" + 0.007*"较" + 0.006*"需求" + 0.006*"在" + 0.006*"为"


In [119]:
documents = df['微博内容分词'].values

In [120]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(1, 0.9750373), (2, 0.024532143)]

Document 2:
[(0, 0.040017076), (1, 0.04001694), (2, 0.04001565), (3, 0.040020455), (4, 0.8399299)]

Document 3:
[(2, 0.13194884), (4, 0.8660745)]

Document 4:
[(1, 0.9993349)]

Document 5:
[(0, 0.05011164), (1, 0.05002069), (2, 0.050019067), (3, 0.79982203), (4, 0.050026596)]

Document 6:
[(0, 0.022437805), (1, 0.022392089), (2, 0.023459509), (3, 0.022443004), (4, 0.90926754)]

Document 7:
[(0, 0.94954234), (1, 0.012576476), (2, 0.012578032), (3, 0.012730111), (4, 0.012573075)]

Document 8:
[(0, 0.100149356), (1, 0.10171494), (2, 0.5954649), (3, 0.10172599), (4, 0.10094487)]

Document 9:
[(3, 0.9722334)]

Document 10:
[(0, 0.9993378)]



### 可视化

In [121]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

In [122]:
# 导出可视化结果到html
# pyLDAvis.save_html(lda_vis, 'lda_visualization.html')

# 二、LDA模型1——铁路与殖民统治

In [196]:
import pandas as p
df = pd.read_excel('railway_construction_AND_colonial_power.xlsx', header=0)

In [197]:
# 打印列名以确认
print(df.columns)

Index(['title', 'href', 'year'], dtype='object')


In [198]:
df.head()

,title,href,year
0,Anticorruption,https://books.google.com/books?hl=zh-CN&lr=&id...,2020
1,"Exploring the Social Life of Japanese"" Manchur...",https://link.springer.com/content/pdf/10.1007/...,2020
2,Comparative media history: An introduction: 17...,https://books.google.com/books?hl=zh-CN&lr=&id...,2005
3,British Imperialism and Decolonization: a Chin...,https://link.springer.com/chapter/10.1057/9781...,2002
4,Skilled labour in colonial economies. Recruitm...,https://www.tandfonline.com/doi/abs/10.1080/00...,2022


In [199]:
len(df)

1000

## 文本预处理

In [200]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) #去除英文停用词

def preprocess_text(text):
    text = text.lower() #小写
    text = re.sub(r'\d+', '', text) #删除数字
    text = re.sub(r'[^\w\s]', '', text) #删除标点符号
    words = text.split() #用空格分割
    words = [word for word in words if word not in stop_words and re.match(r'[a-zA-Z]+', word)]  # 去除停用词和非字母单词
    text = ' '.join(words)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sharon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [201]:
df['title'] = df['title'].astype(str) #将对象转化为字符串类型
df['title_new'] = df['title'].apply(preprocess_text) #执行清洗操作
df['title_new'] = df['title_new'].apply(lambda x: x.split()) #将由空格分割的字符串转化为列表
df.head()


,title,href,year,title_new
0,Anticorruption,https://books.google.com/books?hl=zh-CN&lr=&id...,2020,[anticorruption]
1,"Exploring the Social Life of Japanese"" Manchur...",https://link.springer.com/content/pdf/10.1007/...,2020,"[exploring, social, life, japanese, manchurian..."
2,Comparative media history: An introduction: 17...,https://books.google.com/books?hl=zh-CN&lr=&id...,2005,"[comparative, media, history, introduction, pr..."
3,British Imperialism and Decolonization: a Chin...,https://link.springer.com/chapter/10.1057/9781...,2002,"[british, imperialism, decolonization, chinese..."
4,Skilled labour in colonial economies. Recruitm...,https://www.tandfonline.com/doi/abs/10.1080/00...,2022,"[skilled, labour, colonial, economies, recruit..."


## 主题建模

In [202]:
import gensim.corpora as corpora
from gensim.models import LdaModel
import pyLDAvis 
import pyLDAvis.gensim_models as gensimvis

In [204]:
texts = df['title_new'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [205]:
lda_model = LdaModel(corpus, num_topics= 8, id2word=dictionary, passes=15)  # 训练LDA模型


In [207]:
# 输出每个主题的词语分布
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.019*"colonial" + 0.011*"history" + 0.009*"railway" + 0.009*"social" + 0.009*"city" + 0.009*"century" + 0.007*"town" + 0.007*"nigeria" + 0.006*"war" + 0.006*"urban"')
(1, '0.017*"colonial" + 0.014*"war" + 0.012*"state" + 0.012*"india" + 0.011*"china" + 0.010*"colonialism" + 0.010*"empire" + 0.009*"german" + 0.009*"african" + 0.009*"nation"')
(2, '0.022*"africa" + 0.017*"history" + 0.014*"african" + 0.012*"railway" + 0.011*"chinese" + 0.009*"colonial" + 0.006*"world" + 0.006*"new" + 0.006*"trade" + 0.006*"making"')
(3, '0.020*"colonial" + 0.018*"africa" + 0.011*"world" + 0.011*"development" + 0.009*"british" + 0.008*"political" + 0.008*"economic" + 0.007*"modern" + 0.006*"migration" + 0.006*"south"')
(4, '0.030*"colonial" + 0.014*"africa" + 0.014*"african" + 0.013*"british" + 0.008*"south" + 0.008*"politics" + 0.008*"asia" + 0.008*"economic" + 0.006*"development" + 0.006*"french"')
(5, '0.019*"colonial" + 0.015*"history" + 0.010*"development" + 0.009*"political" + 0.008*"internati

In [211]:
df['title_new'].iloc[0]

['anticorruption']

In [212]:
documents = washed_df['title_new'].values
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(0, 0.013909996), (1, 0.37903818), (2, 0.013910792), (3, 0.013918562), (4, 0.3972569), (5, 0.15412119), (6, 0.013916909), (7, 0.013927453)]

Document 2:
[(0, 0.041685905), (1, 0.70820147), (2, 0.041685395), (3, 0.04167279), (4, 0.041722797), (5, 0.041672792), (6, 0.04168605), (7, 0.04167279)]

Document 3:
[(0, 0.041784976), (1, 0.041671872), (2, 0.70802635), (3, 0.041678283), (4, 0.041707326), (5, 0.041748635), (6, 0.041704547), (7, 0.041678)]

Document 4:
[(0, 0.37961397), (1, 0.017873965), (2, 0.01791827), (3, 0.017899396), (4, 0.017885098), (5, 0.5130554), (6, 0.017883219), (7, 0.01787069)]

Document 5:
[(0, 0.041714642), (1, 0.041689083), (2, 0.04169022), (3, 0.041783556), (4, 0.04168518), (5, 0.70790726), (6, 0.041851126), (7, 0.041678987)]

Document 6:
[(0, 0.010423895), (1, 0.3562452), (2, 0.01042556), (3, 0.010425661), (4, 0.32922605), (5, 0.010423932), (6, 0.010445219), (7, 0.26238444)]

Document 7:
[(0, 0.125), (1, 0.125), (2, 0.125), (3, 0.125), (4, 0.125), (5, 

In [210]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) 
pyLDAvis.display(lda_vis)

# 三、LDA模型2——铁路与反殖民斗争

In [213]:
df = pd.read_excel('colonial_railways_OR_anti-colonial_struggle.xlsx', header=0)

In [214]:
df.head()

,title,href,year
0,Aid in Sino-African Relations: An Analysis of ...,https://search.ebscohost.com/login.aspx?direct...,2011
1,Competing Truths in Northern Ireland,https://digitalcollections.sit.edu/isp_collect...,2007
2,Protests in 21st Century,https://heinonline.org/hol-cgi-bin/get_pdf.cgi...,2020
3,Tracing fragments in the visual culture of ant...,http://shura.shu.ac.uk/id/eprint/10495,2006
4,Cultural accommodation and self-translation in...,https://www.torrossa.com/gs/resourceProxy?an=5...,-1


In [215]:
len(df)

999

## 文本预处理

In [216]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) #去除英文停用词

def preprocess_text(text):
    text = text.lower() #小写
    text = re.sub(r'\d+', '', text) #删除数字
    text = re.sub(r'[^\w\s]', '', text) #删除标点符号
    words = text.split() #用空格分割
    words = [word for word in words if word not in stop_words and re.match(r'[a-zA-Z]+', word)]  # 去除停用词和非字母单词
    text = ' '.join(words)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sharon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [217]:
df['title'] = df['title'].astype(str) #将对象转化为字符串类型
df['title_new'] = df['title'].apply(preprocess_text) #执行清洗操作
df['title_new'] = df['title_new'].apply(lambda x: x.split()) #将由空格分割的字符串转化为列表
df.head()

,title,href,year,title_new
0,Aid in Sino-African Relations: An Analysis of ...,https://search.ebscohost.com/login.aspx?direct...,2011,"[aid, sinoafrican, relations, analysis, prompt..."
1,Competing Truths in Northern Ireland,https://digitalcollections.sit.edu/isp_collect...,2007,"[competing, truths, northern, ireland]"
2,Protests in 21st Century,https://heinonline.org/hol-cgi-bin/get_pdf.cgi...,2020,"[protests, st, century]"
3,Tracing fragments in the visual culture of ant...,http://shura.shu.ac.uk/id/eprint/10495,2006,"[tracing, fragments, visual, culture, anticolo..."
4,Cultural accommodation and self-translation in...,https://www.torrossa.com/gs/resourceProxy?an=5...,-1,"[cultural, accommodation, selftranslation, wor..."


## 主题建模

In [218]:
import gensim.corpora as corpora
from gensim.models import LdaModel
import pyLDAvis 
import pyLDAvis.gensim_models as gensimvis

In [219]:
texts = df['title_new'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [220]:
lda_model = LdaModel(corpus, num_topics= 8, id2word=dictionary, passes=15)  # 训练LDA模型

In [221]:
# 输出每个主题的词语分布
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.014*"africa" + 0.010*"political" + 0.007*"resistance" + 0.006*"postcolonial" + 0.006*"state" + 0.006*"violence" + 0.005*"women" + 0.005*"nationalism" + 0.005*"memory" + 0.005*"development"')
(1, '0.018*"india" + 0.008*"indian" + 0.008*"struggle" + 0.008*"society" + 0.008*"civil" + 0.007*"anticolonial" + 0.007*"africa" + 0.007*"politics" + 0.007*"violence" + 0.007*"nationalist"')
(2, '0.009*"african" + 0.008*"history" + 0.008*"cyprus" + 0.007*"revolution" + 0.006*"struggle" + 0.005*"colonial" + 0.005*"politics" + 0.005*"anticolonial" + 0.004*"war" + 0.004*"postcolonial"')
(3, '0.014*"struggle" + 0.013*"anticolonial" + 0.009*"colonial" + 0.009*"revolution" + 0.009*"frantz" + 0.009*"nationalism" + 0.008*"fanon" + 0.008*"world" + 0.007*"postcolonial" + 0.006*"empire"')
(4, '0.024*"anticolonial" + 0.022*"struggle" + 0.014*"politics" + 0.006*"african" + 0.006*"islamic" + 0.006*"study" + 0.005*"india" + 0.005*"feminism" + 0.005*"zimbabwe" + 0.005*"postcolonial"')
(5, '0.015*"colonial" 

In [222]:
df['title_new'].iloc[0]

['aid',
 'sinoafrican',
 'relations',
 'analysis',
 'promptings',
 'pluses',
 'problems',
 'chinas',
 'aid',
 'africa']

In [223]:
documents = washed_df['title_new'].values
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(0, 0.9204255), (1, 0.011373713), (2, 0.011368499), (3, 0.01136594), (4, 0.011365711), (5, 0.011365541), (6, 0.011368665), (7, 0.011366515)]

Document 2:
[(0, 0.025003543), (1, 0.8249442), (2, 0.025020193), (3, 0.025006661), (4, 0.025003543), (5, 0.025006583), (6, 0.025006605), (7, 0.025008649)]

Document 3:
[(0, 0.031253707), (1, 0.78119445), (2, 0.031259444), (3, 0.03125899), (4, 0.031253707), (5, 0.031258855), (6, 0.031253707), (7, 0.03126716)]

Document 4:
[(0, 0.89043146), (1, 0.015648484), (2, 0.015641311), (3, 0.01566445), (4, 0.01568477), (5, 0.01563386), (6, 0.015660714), (7, 0.01563495)]

Document 5:
[(0, 0.015641317), (1, 0.015637081), (2, 0.89053494), (3, 0.01562792), (4, 0.015638236), (5, 0.01563798), (6, 0.015639903), (7, 0.015642662)]

Document 6:
[(3, 0.9326209)]

Document 7:
[(0, 0.02503702), (1, 0.0250057), (2, 0.0250057), (3, 0.02501859), (4, 0.0250057), (5, 0.8248899), (6, 0.025018336), (7, 0.025019007)]

Document 8:
[(0, 0.017901182), (1, 0.01792493), 

In [224]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) 
pyLDAvis.display(lda_vis)